In [ ]:
using Zygote
using JuMP
using Ipopt
using LinearAlgebra
using MathOptInterface
const MOI = MathOptInterface
using SparseArrays
using BenchmarkTools
using Plots

# Feasibility restoration phase function

In [ ]:
function restoration_phase(xk, ρ)
    println("Solving restoration problem")
    
    is_s_eq_0 = false
    
    # Nonlinear model
    nlrp = Model()
    set_silent(nlrp)
    @variable(nlrp, x[i=1:2])
    @NLobjective(nlrp, Min, (1 - x[1])^2 + 100*(x[2] - x[1]^2)^2)
    @NLconstraint(nlrp, c2, x[1]^2 + x[2]^2 - 2 <= 0)
    @NLconstraint(nlrp, c3, -1.5 - x[1] <= 0)
    @NLconstraint(nlrp, c4, x[1] - 1.5 <= 0)
    @NLconstraint(nlrp, c5, -1.5 - x[2] <= 0)
    @NLconstraint(nlrp, c6, x[2] - 1.5 <= 0)
    
    # Initialise MOI evaluator
    pp = NLPEvaluator(nlrp)
    MOI.initialize(pp, [:Jac,:Grad,:Hess])

    while !is_s_eq_0
        
        #Constraint eval
        ggg=zeros(length(all_nonlinear_constraints(nlrp)))
        MOI.eval_constraint(pp,ggg,xk)

        #Jacobian eval
        JStru=MOI.jacobian_structure(pp)
        Ja=zeros(length(JStru))
        MOI.eval_constraint_jacobian(pp,Ja,xk)

        jr = Vector{Int64}(undef, length(Ja)); jc = Vector{Int64}(undef, length(Ja))
        for i in 1:length(JStru)
            jr[i] = JStru[i][1] |> Int
            jc[i] = JStru[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, Ja) 
        Jacobi = Matrix(Jacobeval)
        
        restor_model = Model(Ipopt.Optimizer)
        set_silent(restor_model)
        @variable(restor_model, s >= 0)
        @variable(restor_model, d[i=1:2])

        @objective(restor_model, Min, s) 
        @constraint(restor_model, c[i in 1:length(all_nonlinear_constraints(nlrp))],  ggg[i] + dot(Jacobi[i, :], d) - s <= 0)
        @constraint(restor_model, c7, d[1] <= ρ)
        @constraint(restor_model, c8, d[2] <= ρ) 
        @constraint(restor_model, c9, d[1] >= -ρ)
        @constraint(restor_model, c10, d[2] >= -ρ)
        optimize!(restor_model)

        proposed_xk = xk .+ value.(d)
        
        # merit function
        function m(x::Vector{<:Real}) 
            ce = zeros(length(all_nonlinear_constraints(nlrp)))
            MOI.eval_constraint(pp,ce,x)
            
            max_ineq_rows = []
            for i in 1:length(ce)
                maxval = max(ce[i], 0)
                max_ineq_rows = push!(max_ineq_rows, maxval)
            end
            
            merit = norm(max_ineq_rows)
            return merit
        end

        # Update step
        if abs(value(s)) < 1e-6
        elseif m(proposed_xk) < m(xk) 
            println("restoration merit condition: success")
            xk = proposed_xk
            ρ += 1*ρ
        else
            println("restoration merit condition: failed")
            ρ -= 0.5*ρ
        end
        
#         println("slack = $(value(s))")
        is_s_eq_0 = abs(value(s)) < 1e-6
    end
    
    return xk, ρ 
end

# SLP with a merit function

In [ ]:
function SLP_MF(xk::Vector{<:Real}, ρ, γ)
    # ρ and γ are parameters
    @assert γ>=0
    
    has_x_converged = false
    ctr = 0
    restor_ctr = 0
    
    # Nonlinear model
    nlmod = Model()
    set_silent(nlmod)
    @variable(nlmod, x[i=1:2])
    @NLobjective(nlmod, Min, (1 - x[1])^2 + 100*(x[2] - x[1]^2)^2)
    @NLconstraint(nlmod, c2, x[1]^2 + x[2]^2 - 2 <= 0)
    @NLconstraint(nlmod, c3, -1.5 - x[1] <= 0)
    @NLconstraint(nlmod, c4, x[1] - 1.5 <= 0)
    @NLconstraint(nlmod, c5, -1.5 - x[2] <= 0)
    @NLconstraint(nlmod, c6, x[2] - 1.5 <= 0)
    
    # Initialise MOI evaluator
    p = NLPEvaluator(nlmod)
    MOI.initialize(p, [:Jac,:Grad,:Hess])
    
    # Iterate until convergence
    while has_x_converged != [1, 1]
        ctr += 1
        
        #Objective eval
        objEv=MOI.eval_objective(p, xk)

        #Objective gradient
        df=zeros(length(xk))
        MOI.eval_objective_gradient(p,df,xk)

        #Constraint eval
        gg=zeros(length(all_nonlinear_constraints(nlmod)))
        MOI.eval_constraint(p,gg,xk)

        #Jacobian eval
        JStr=MOI.jacobian_structure(p)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(p,J,xk)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        Jacobi = Matrix(Jacobeval)

        # Linearised model
        model = Model(Ipopt.Optimizer)
        set_silent(model)
        @variable(model, d[i=1:2])
        @objective(model, Min, objEv + df'*d)
        @constraint(model, c[i in 1:length(all_nonlinear_constraints(nlmod))],  gg[i] + dot(Jacobi[i, :], d) <= 0)
        @constraint(model, c7, d[1] <= ρ)
        @constraint(model, c8, d[2] <= ρ) 
        @constraint(model, c9, d[1] >= -ρ)
        @constraint(model, c10, d[2] >= -ρ)
        
        println("Iteration $ctr...")

        optimize!(model)
        @show termination_status(model)
        @show objective_value(model)
        
        # updating ρ
        proposed_xk = xk .+ value.(d)
        
        # merit function
        function m(x::Vector) 
            newobjev = MOI.eval_objective(p, x)
            ce = zeros(length(all_nonlinear_constraints(nlmod)))
            MOI.eval_constraint(p,ce,x)
            
            max_ineq_rows = []
            for i in 1:length(ce)
                maxval = max(ce[i], 0)
                max_ineq_rows = push!(max_ineq_rows, maxval)
            end
            
            merit = newobjev + γ * norm(max_ineq_rows)
            return merit
        end
        
        println("m(xk) = $(m(xk))")
        println("m(prop_xk) = $(m(proposed_xk))")
        
        # Update step
        if termination_status(model) == MathOptInterface.LOCALLY_INFEASIBLE
        elseif m(proposed_xk) < m(xk)
            println("merit condition: success")
            xk = proposed_xk
            ρ += 1*ρ
        else
            println("merit condition: failed")
            ρ -= 0.5*ρ
        end
        
        println("d=$(value.(d))")
        println("x$(ctr)=$(xk)")
#         println("new ρ: $ρ")
        
        #restoration phase:
        if termination_status(model) == MathOptInterface.LOCALLY_INFEASIBLE
            xk, ρ = restoration_phase(xk, ρ)
            restor_ctr += 1
        end
        
        #convergence test
        tol = 1e-7
        has_x_converged = abs.(value.(d)) .< tol
        
    end
    println("no. restoration phases: $(restor_ctr)")
end

In [ ]:
SLP_MF([100., -100.], 4, 1)

# SLP with a filter function

In [ ]:
function SLP_F(xk::Vector{<:Real}, ρ)
    # ρ is the trust region parameter

    has_x_converged = false
    ctr = 0
    restor_ctr = 0
    
    filt = []
    
    # Nonlinear model
    nlmod = Model()
    set_silent(nlmod)
    @variable(nlmod, x[i=1:2])
    @NLobjective(nlmod, Min, (1 - x[1])^2 + 100*(x[2] - x[1]^2)^2)
    @NLconstraint(nlmod, c2, x[1]^2 + x[2]^2 - 2 <= 0)
    @NLconstraint(nlmod, c3, -1.5 - x[1] <= 0)
    @NLconstraint(nlmod, c4, x[1] - 1.5 <= 0)
    @NLconstraint(nlmod, c5, -1.5 - x[2] <= 0)
    @NLconstraint(nlmod, c6, x[2] - 1.5 <= 0)
    
    #Initialise MOI evaluator
    p = NLPEvaluator(nlmod)
    MOI.initialize(p, [:Jac,:Grad,:Hess])
    
    # Iterate until convergence
    while has_x_converged != [1, 1]
        ctr += 1
        
        p = NLPEvaluator(nlmod)
        MOI.initialize(p, [:Jac,:Grad,:Hess])
        
        #Objective eval
        objEv=MOI.eval_objective(p, xk)

        #Objective gradient
        df=zeros(length(xk))
        MOI.eval_objective_gradient(p,df,xk)

        #Constraint eval
        gg=zeros(length(all_nonlinear_constraints(nlmod)))
        MOI.eval_constraint(p,gg,xk)

        #Jacobian eval
        JStr=MOI.jacobian_structure(p)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(p,J,xk)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        Jacobi = Matrix(Jacobeval)
        
        # Linearised model
        model = Model(Ipopt.Optimizer)
        set_silent(model)
        @variable(model, d[i=1:2])
        @objective(model, Min, objEv + df'*d)
        @constraint(model, c[i in 1:length(all_nonlinear_constraints(nlmod))],  gg[i] + dot(Jacobi[i, :], d) <= 0)
        @constraint(model, c7, d[1] <= ρ)
        @constraint(model, c8, d[2] <= ρ) 
        @constraint(model, c9, d[1] >= -ρ)
        @constraint(model, c10, d[2] >= -ρ)
        
        println("Iteration $ctr...")

        optimize!(model)
        @show termination_status(model)
        
        #functions for making the filter
        function fk(x::Vector{<:Real}) 
            newobjev = MOI.eval_objective(p, x)
            return newobjev
        end
        function hk(x::Vector{<:Real}) 
            ce = zeros(length(all_nonlinear_constraints(nlmod)))
            MOI.eval_constraint(p,ce,x)
            
            max_ineq_rows = []
            for i in 1:length(ce)
                maxval = max(ce[i], 0)
                max_ineq_rows = push!(max_ineq_rows, maxval)
            end
            
            h = norm(max_ineq_rows)
            return h
        end
        
        proposed_xk = xk .+ value.(d)
        better_obj = all(v -> fk(proposed_xk) < v[1], filt)
        better_constr = all(v -> hk(proposed_xk) < v[2], filt)
        
        # Update step
        if termination_status(model) == MathOptInterface.LOCALLY_INFEASIBLE
        elseif better_obj || better_constr
            println("filter: accept")
            xk = proposed_xk
            
            # trust region update
            ρ += 1*ρ
            filt = push!(filt, (fk(xk), hk(xk)))
        else
            println("filter: reject")
            
            # trust region update
            ρ -= 0.5*ρ
#             max_step = norm(value.(d), Inf) # more aggressive trust region update
#             ρ = max_step/3
            filt = push!(filt, (fk(proposed_xk), hk(proposed_xk)))
        end
        
        
        println("f(propxk) = $(fk(proposed_xk))")
        println("d=$(value.(d))")
        println("x$(ctr)=$xk")  
        
        #restoration phase:
        if termination_status(model) == MathOptInterface.LOCALLY_INFEASIBLE
            xk, ρ = restoration_phase(xk, ρ)
            restor_ctr += 1
        end
    
        #convergence test
        tol = 1e-5
        has_x_converged = abs.(value.(d)) .< tol
        
    end
    println("no. restoration phases: $(restor_ctr)")
end


In [ ]:
SLP_F([100., -100.], 4)

# SQP with a merit function

In [ ]:
function SQP_MF(xk::Vector{<:Real}, λk::Vector{Float64}, ρ, γ)
    # λ, μ are initial lagrangian multipliers
    # ρ and γ are parameters

    has_x_converged = false
    ctr = 0
    restor_ctr = 0
    
    # Nonlinear model
    nlmod = Model(Ipopt.Optimizer)
    set_silent(nlmod)
    @variable(nlmod, x[i=1:2])
    @NLobjective(nlmod, Min, (1 - x[1])^2 + 100*(x[2] - x[1]^2)^2)
    @NLconstraint(nlmod, c2, x[1]^2 + x[2]^2 - 2 <= 0)
    @NLconstraint(nlmod, c3, -1.5 - x[1] <= 0)
    @NLconstraint(nlmod, c4, x[1] - 1.5 <= 0)
    @NLconstraint(nlmod, c5, -1.5 - x[2] <= 0)
    @NLconstraint(nlmod, c6, x[2] - 1.5 <= 0)
    p = NLPEvaluator(nlmod)
    MOI.initialize(p, [:Jac,:Grad,:Hess])
    
    
    while has_x_converged != [1, 1]
        ctr += 1
        
        p = NLPEvaluator(nlmod)
        MOI.initialize(p, [:Jac,:Grad,:Hess])
        
        #Objective eval
        objEv=MOI.eval_objective(p, xk)

        #Objective gradient
        df=zeros(length(xk))
        MOI.eval_objective_gradient(p,df,xk)

        #Constraint eval
        gg=zeros(length(all_nonlinear_constraints(nlmod)))
        MOI.eval_constraint(p,gg,xk)

        #Jacobian eval
        JStr=MOI.jacobian_structure(p)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(p,J,xk)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        Jacobi = Matrix(Jacobeval)
        
        #Hessian-of-the-Lagrangian eval
        HStr=MOI.hessian_lagrangian_structure(p)
        H=zeros(length(HStr))
        MOI.eval_hessian_lagrangian(p,H,xk,1.0,λk) 
        
        hr = Vector{Int64}(undef, length(H)); hc = Vector{Int64}(undef, length(H))
        for i in 1:length(HStr)
            hr[i] = HStr[i][1] |> Int
            hc[i] = HStr[i][2] |> Int
        end
        
        HessLag = sparse(hr, hc, H)
        
        hessianobj(d) = 0.5*d'*(HessLag*d)
        
        # Linear model
        model = Model(Ipopt.Optimizer)
        set_silent(model)
        @variable(model, d[i=1:2])
        @objective(model, Min, objEv + df'*d + hessianobj(d)) 
        @constraint(model, c[i in 1:length(all_nonlinear_constraints(nlmod))],  gg[i] + dot(Jacobi[i, :], d) <= 0)
        @constraint(model, c7, d[1] <= ρ)
        @constraint(model, c8, d[2] <= ρ) 
        @constraint(model, c9, d[1] >= -ρ)
        @constraint(model, c10, d[2] >= -ρ)
        
        println("Iteration $ctr...")
        
        optimize!(model)
        @show termination_status(model)
        
        # update λ
        λk = []
        for i in 1:length(all_nonlinear_constraints(nlmod))
            dual = shadow_price(c[i])
            λk = push!(λk, dual)
        end
        λk = convert(Vector{Float64},λk)
        

        # merit function
        function m(x::Vector{<:Real}) 
            newobjev = MOI.eval_objective(p, x)
            ce = zeros(length(all_nonlinear_constraints(nlmod)))
            MOI.eval_constraint(p,ce,x)
            
            max_ineq_rows = []
            for i in 1:length(ce)
                maxval = max(ce[i], 0)
                max_ineq_rows = push!(max_ineq_rows, maxval)
            end
            
            merit = newobjev + γ * norm(max_ineq_rows)
            return merit
        end
        
        # update step
        proposed_xk = xk .+ value.(d)
        println("m(xk) = $(m(xk))")
        println("m(prop_xk) = $(m(proposed_xk))")
        
        if termination_status(model) == MathOptInterface.LOCALLY_INFEASIBLE
        elseif m(proposed_xk) < m(xk)
            println("merit condition: success")
            xk = proposed_xk
            ρ += 1*ρ
        else
            println("merit condition: failed")
            ρ -= 0.5*ρ
        end
        
        println("d=$(value.(d))")
        println("x$ctr=$xk")
        
        #restoration phase:
        if termination_status(model) == MathOptInterface.LOCALLY_INFEASIBLE
            xk, ρ = restoration_phase(xk, ρ)
            restor_ctr += 1
        end
        
        #convergence test
        tol = 1e-6
        has_x_converged = abs.(value.(d)) .< tol
    end
    println("no. restoration phases: $(restor_ctr)")
end

In [ ]:
SQP_MF([-10., 20.], [0., 0., 0., 0., 0., 0.], 4, 5)

# SQP with a filter function

In [ ]:
function SQP_F(xk::Vector{<:Real}, λk::Vector{Float64}, ρ)
    # λ is initial lagrangian multiplier
    # ρ is the trust region parameter

    has_x_converged = false
    ctr = 0
    restor_ctr = 0
    
    filt = []
    
    # Nonlinear model
    nlmod = Model(Ipopt.Optimizer)
    @variable(nlmod, x[i=1:2])
    @NLobjective(nlmod, Min, (1 - x[1])^2 + 100*(x[2] - x[1]^2)^2)
    @NLconstraint(nlmod, c2, x[1]^2 + x[2]^2 - 2 <= 0)
    @NLconstraint(nlmod, c3, -1.5 - x[1] <= 0)
    @NLconstraint(nlmod, c4, x[1] - 1.5 <= 0)
    @NLconstraint(nlmod, c5, -1.5 - x[2] <= 0)
    @NLconstraint(nlmod, c6, x[2] - 1.5 <= 0)
    p = NLPEvaluator(nlmod)
    MOI.initialize(p, [:Jac,:Grad,:Hess])
    
    
    while has_x_converged != [1, 1]
        ctr += 1
        
        p = NLPEvaluator(nlmod)
        MOI.initialize(p, [:Jac,:Grad,:Hess])
        
        #Objective eval
        objEv=MOI.eval_objective(p, xk)

        #Objective gradient
        df=zeros(length(xk))
        MOI.eval_objective_gradient(p,df,xk)

        #Constraint eval
        gg=zeros(length(all_nonlinear_constraints(nlmod)))
        MOI.eval_constraint(p,gg,xk)

        #Jacobian eval
        JStr=MOI.jacobian_structure(p)
        J=zeros(length(JStr))
        MOI.eval_constraint_jacobian(p,J,xk)

        jr = Vector{Int64}(undef, length(J)); jc = Vector{Int64}(undef, length(J))
        for i in 1:length(JStr)
            jr[i] = JStr[i][1] |> Int
            jc[i] = JStr[i][2] |> Int
        end
        Jacobeval = sparse(jr, jc, J) 
        Jacobi = Matrix(Jacobeval)
        
        #Hessian-of-the-Lagrangian eval
        HStr=MOI.hessian_lagrangian_structure(p)
        H=zeros(length(HStr))
        MOI.eval_hessian_lagrangian(p,H,xk,1.0,λk) 
        
        hr = Vector{Int64}(undef, length(H)); hc = Vector{Int64}(undef, length(H))
        for i in 1:length(HStr)
            hr[i] = HStr[i][1] |> Int
            hc[i] = HStr[i][2] |> Int
        end
        HessLag = sparse(hr, hc, H)
        
        hessianobj(d) = 0.5*d'*(HessLag*d)
        
        # Linear model
        model = Model(Ipopt.Optimizer)
        set_silent(model)
        @variable(model, d[i=1:2])
        @objective(model, Min, objEv + df'*d + hessianobj(d)) 
        @constraint(model, c[i in 1:length(all_nonlinear_constraints(nlmod))],  gg[i] + dot(Jacobi[i, :], d) <= 0)
        @constraint(model, c7, d[1] <= ρ)
        @constraint(model, c8, d[2] <= ρ) 
        @constraint(model, c9, d[1] >= -ρ)
        @constraint(model, c10, d[2] >= -ρ)

        optimize!(model)
        @show termination_status(model)
                
        println("Iteration $ctr...")
        
        # update λ
        λk = []
        for i in 1:length(all_nonlinear_constraints(nlmod))
            dual = shadow_price(c[i])
            λk = push!(λk, dual)
        end
        λk = convert(Vector{Float64},λk)

        #functions for making the filter
        function fk(x::Vector{<:Real}) 
            newobjev = MOI.eval_objective(p, x)
            return newobjev
        end
        function hk(x::Vector{<:Real}) 
            ce = zeros(length(all_nonlinear_constraints(nlmod)))
            MOI.eval_constraint(p,ce,x)
            
            max_ineq_rows = []
            for i in 1:length(ce)
                maxval = max(ce[i], 0)
                max_ineq_rows = push!(max_ineq_rows, maxval)
            end
            
            h = norm(max_ineq_rows)
            return h
        end
        
        # update step
        proposed_xk = xk .+ value.(d)
        better_obj = all(v -> fk(proposed_xk) < v[1], filt)
        better_constr = all(v -> hk(proposed_xk) < v[2], filt)
        
        if termination_status(model) == MathOptInterface.LOCALLY_INFEASIBLE
        elseif better_obj || better_constr
            println("filter: accept")
            xk = proposed_xk
            ρ += 1*ρ
            filter = push!(filt, (fk(xk), hk(xk)))
        else
            println("filter: reject")
            ρ -= 0.5*ρ
            filter = push!(filt, (fk(proposed_xk), hk(proposed_xk)))
        end
        
        
        println("f(propxk) = $(fk(proposed_xk))")
        println("d=$(value.(d))")
        println("x$ctr=$xk")
        
        #restoration phase:
        if termination_status(model) == MathOptInterface.LOCALLY_INFEASIBLE
            xk, ρ = restoration_phase(xk, ρ)
            restor_ctr += 1
        end
        
        #convergence test
        tol = 1e-7
        has_x_converged = abs.(value.(d)) .< tol
    end
    println("no. restoration phases: $(restor_ctr)")
end

In [ ]:
SQP_F([-10., 20.], [0.,0.,0.,0.,0.,0.], 4)

# Plotting functions:

In [ ]:
using Plots
function rosen_disk(x,y)
    if (x)^2 + y^2 > 2
        return NaN
    end
    
    return (1-x)^2 + 100*(y - x^2)^2
end

x = -1.5:0.005:1.5
y = -1.5:0.005:1.5

rosen_disk_plot = Plots.heatmap(x,y,(x,y)->rosen_disk(x,y),c=:jet, title="Disk-Constrained Rosenbrock Function", xlabel = "x", ylabel = "y", aspect_ratio=:equal)

In [ ]:
scatter!(rosen_disk_plot, [1], [1], markershape = :circle, label = "Global")